In [8]:
#数据部分
import numpy as np
import os, nltk
import xml.etree.ElementTree as ET

ddi_db_2013_train_xml_folder = 'xmlformat/Train2013/DrugBank'
ddi_db_2013_test_xml_folder = 'xmlformat/Test2013/DrugBank'
ddi_ml_2013_train_xml_folder = 'xmlformat/Train2013/MedLine'
ddi_ml_2013_test_xml_folder = 'xmlformat/Test2013/MedLine'

ddi_2011_train_xml_folder = 'xmlformat/Train2011'
ddi_2011_test_xml_folder = 'xmlformat/Test2011'

EMBEDDING_DIM = 400

ddi_2013_all_train = '2013_all_train.txt'
ddi_2013_all_train = '2013_all_train.txt'
ddi_2013_DB_train = '2013_DB_train.txt'
ddi_2013_ML_train = '2013_ML_train.txt'
ddi_2013_DB_test = '2013_DB_test.txt'
ddi_2013_ML_test = '2013_ML_test.txt'

ddi_2011_train = '2011_train.txt'
ddi_2011_test = '2011_test.txt'

ddi_2011_sentences = '2011_sentences.txt'
ddi_2013_sentences = '2013_sentences.txt'

ddi_2011_gold = 'gold annotations test.txt'

pad = 'NULL'
r = 3

special_chars_13 = [';', '!', '?', ',', ';', '*', '\\', '$', '%', '`', '~', '@', '^', '&', ':', '"', '<', '>']
special_chars_11 = [';', '!', '?', ',', ';', '*', '\\', '$', '%', '`', '~', '@', '^', '&', ':', '"', '<', '>', '/', '(',
                    ')', '[', ']', '-']

DDIE_2011_gold_standard = {}


###
### 数据预处理部分
###

def get_ranged_indices(str_indexs):
    if str_indexs[0] == '[':
        str_indexs = str_indexs[1:]
    elif str_indexs.startswith('range('):
        str_indexs = str_indexs[6:]
    if str_indexs[-1] == ']' or str_indexs[-1] == ')':
        str_indexs = str_indexs[:-1]
    return [int(i.strip()) for i in str_indexs.split(',')]


def read_corpus_new(text_file):
    instances = []
    for line in open(text_file):
        instance = line.strip().split('\t')
        if len(instance) == 4:
            target, sentence, d1i, d2i = instance
            instances.append((target, sentence, get_ranged_indices(d1i), get_ranged_indices(d2i)))
        elif len(instance) == 5:
            target, sentence, d1i, d2i, pairid = instance
            instances.append((target, sentence, get_ranged_indices(d1i), get_ranged_indices(d2i), pairid))
    return instances


def get_entity_by_offset(sentence, offset, plus_one=True):
    if type(offset[0]) == tuple:
        origin_entity = ' '.join([sentence[x[0]:x[1] + (1 if plus_one else 0)] for x in offset])
        return origin_entity
    else:
        origin_entity = sentence[offset[0]:offset[1] + (1 if plus_one else 0)]
        return origin_entity


def get_entity_by_range(sentence, r):
    return ' '.join(sentence.split(' ')[i] for i in r)


def count_pair(e1, e2, counter, target, identical_test):
    e1 = e1.replace(' ', '').replace('-', '')
    e2 = e2.replace(' ', '').replace('-', '')
    key1 = e1 + '_' + e2
    key2 = e2 + '_' + e1
    if key1 in counter:
        counter[key1] += 1
        if identical_test[key1] != target:
            print(e1, e2, "have different labels")
        return key1
    elif key2 in counter:
        counter[key2] += 1
        if identical_test[key2] != target:
            print(e1, e2, "have different labels")
        return key2
    else:
        counter[key1] = 1
        identical_test[key1] = target
        return key1


def label21hot(label, dict):
    vector = [0] * len(dict)
    vector[dict[label]] = 1
    return vector


def get_distance(idx, indices):
    dis = [idx - x for x in indices]
    return dis[np.argmin(np.abs(dis))]


def summarize_ddi_instance(corpora_files=(ddi_2013_DB_train, ddi_2013_ML_train, ddi_2013_DB_test, ddi_2013_ML_test)):
    DB_train_set = read_corpus_new(corpora_files[0])
    ML_train_set = read_corpus_new(corpora_files[1])
    DB_test_set = read_corpus_new(corpora_files[2])
    ML_test_set = read_corpus_new(corpora_files[3])

    train_set = DB_train_set + ML_train_set
    test_set = DB_test_set + ML_test_set

    train_counter = {}
    test_counter = {}

    identical_test = {}

    train_ddi_list = []
    test_ddi_list = []

    for target, sentence, r1, r2 in train_set:
        e1 = get_entity_by_range(sentence, r1)
        e2 = get_entity_by_range(sentence, r2)
        train_ddi_list.append(count_pair(e1, e2, train_counter, target, identical_test))
        if e1 == 'BREVIBLOC' and e2 == 'digoxin':
            print(sentence)

    for target, sentence, r1, r2 in test_set:
        e1 = get_entity_by_range(sentence, r1)
        e2 = get_entity_by_range(sentence, r2)
        test_ddi_list.append(count_pair(e1, e2, test_counter, target, identical_test))
        if e1 == 'BREVIBLOC' and e2 == 'digoxin':
            print(sentence)
    exit(0)
    import operator
    print(sorted(train_counter.items(), key=operator.itemgetter(1)))

    print(sum(train_counter.values()), list(train_counter.values()).count(1))
    print(sorted(test_counter.items(), key=operator.itemgetter(1)))
    print(sum(test_counter.values()), list(test_counter.values()).count(1))
    print(max(train_counter.values()), max(test_counter.values()))
    fd = nltk.FreqDist(train_ddi_list)
    fd.plot(30, cumulative=False)
    fd = nltk.FreqDist(test_ddi_list)
    fd.plot(30, cumulative=False)


def load_data():
    #############
    # LOAD DATA #
    #############
    lower_case = False
    train_counter = {}
    test_counter = {}
    target_indices2013 = {}
    target_indices2011 = {}

    DB_train_set2013 = read_corpus_new(ddi_2013_DB_train)
    ML_train_set2013 = read_corpus_new(ddi_2013_ML_train)
    DB_test_set2013 = read_corpus_new(ddi_2013_DB_test)
    ML_test_set2013 = read_corpus_new(ddi_2013_ML_test)
    train_set2013 = DB_train_set2013 + ML_train_set2013
    test_set2013 = DB_test_set2013 + ML_test_set2013

    train_set2011 = read_corpus_new(ddi_2011_train)
    test_set2011 = read_corpus_new(ddi_2011_test)

    target_indices2013['-1'] = 0
    target_indices2013['mechanism'] = 1
    target_indices2013['effect'] = 2
    target_indices2013['advise'] = 3
    target_indices2013['int'] = 4

    target_indices2011['0'] = 0
    target_indices2011['1'] = 1

    word_indices = {}  # word:index
    word_indices[pad] = 0
    dis_indices = {}
    word_index = 1
    dis_index = 1
    for _, sentence, e1r, e2r in train_set2011 + test_set2011 + train_set2013 + test_set2013:
        token_seq = sentence.split(' ')
        for i, word in enumerate(token_seq):
            if lower_case:
                word = word.lower()
            d1 = get_distance(i, e1r)
            d2 = get_distance(i, e2r)
            if d1 not in dis_indices:
                dis_indices[d1] = dis_index
                dis_index += 1
            if d2 not in dis_indices:
                dis_indices[d2] = dis_index
                dis_index += 1
            if word not in word_indices:
                word_indices[word] = word_index
                word_index += 1

    def get_input_format(train_set, test_set, target_indices):
        X_train_text = []
        X_train_inner_tokens = []
        X_train_surrounding_tokens = []
        X_train_e1 = []
        X_train_e2 = []
        X_train_distance1 = []
        X_train_distance2 = []
        X_train_skeleton = []
        y_train = []
        X_test_text = []
        X_test_inner_tokens = []
        X_test_surrounding_tokens = []
        X_test_e1 = []
        X_test_e2 = []
        X_test_distance1 = []
        X_test_distance2 = []
        X_test_skeleton = []
        y_test = []
        for target, sentence, e1r, e2r in train_set:
            token_seq = sentence.split(' ')
            X_train_text.append([word_indices[word.lower() if lower_case else word] for word in token_seq])
            X_train_distance1.append([dis_indices[get_distance(i, e1r)] for i in range(len(token_seq))])
            X_train_distance2.append([dis_indices[get_distance(i, e2r)] for i in range(len(token_seq))])
            y_train.append(label21hot(target, target_indices))

            skeleton_cache = [0] * len(token_seq)
            cache = []
            for i in range(max(e1r) + 1, min(e2r)):
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 1
            if len(X_train_inner_tokens) == 0:
                cache.append(word_indices[pad])
            X_train_inner_tokens.append(cache)

            cache = []
            for i in e1r:
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 2
            X_train_e1.append(cache)

            cache = []
            for i in e2r:
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 3
            X_train_e2.append(cache)

            cache = []
            for i in list(range(min(e1r) - r + 1, min(e1r))) + list(range(max(e1r) + 1, max(e1r) + r)) + list(range(
                                    min(e2r) - r + 1, min(e2r))) + list(range(max(e2r) + 1, max(e2r) + r)):
                if i < 0 or i > len(token_seq) - 1:
                    cache.append(word_indices[pad])
                else:
                    cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                    if skeleton_cache[i] != 0:
                        skeleton_cache[i] = 4
                    else:
                        skeleton_cache[i] = 5
            X_train_skeleton.append(skeleton_cache)
            X_train_surrounding_tokens.append(cache)

        for target, sentence, e1r, e2r in test_set:
            token_seq = sentence.split(' ')
            X_test_text.append([word_indices[word.lower() if lower_case else word] for word in token_seq])
            X_test_distance1.append([dis_indices[get_distance(i, e1r)] for i in range(len(token_seq))])
            X_test_distance2.append([dis_indices[get_distance(i, e2r)] for i in range(len(token_seq))])
            y_test.append(label21hot(target, target_indices))

            skeleton_cache = [0] * len(token_seq)
            cache = []
            for i in range(max(e1r) + 1, min(e2r)):
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 1
            if len(X_test_inner_tokens) == 0:
                cache.append(word_indices[pad])
            X_test_inner_tokens.append(cache)

            cache = []
            for i in e1r:
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 2
            X_test_e1.append(cache)

            cache = []
            for i in e2r:
                cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                skeleton_cache[i] = 3
            X_test_e2.append(cache)

            cache = []
            for i in list(range(min(e1r) - r + 1, min(e1r))) + list(range(max(e1r) + 1, max(e1r) + r)) + list(range(
                                    min(e2r) - r + 1, min(e2r))) + list(range(max(e2r) + 1, max(e2r) + r)):
                if i < 0 or i > len(token_seq) - 1:
                    cache.append(word_indices[pad])
                else:
                    cache.append(word_indices[token_seq[i].lower() if lower_case else token_seq[i]])
                    if skeleton_cache[i] != 0:
                        skeleton_cache[i] = 4
                    else:
                        skeleton_cache[i] = 5
            X_test_skeleton.append(skeleton_cache)
            X_test_surrounding_tokens.append(cache)

        X_train_text = np.asarray(X_train_text)
        X_train_inner_tokens = np.asarray(X_train_inner_tokens)
        X_train_surrounding_tokens = np.asarray(X_train_surrounding_tokens)
        X_train_e1 = np.asarray(X_train_e1)
        X_train_e2 = np.asarray(X_train_e2)
        X_train_distance1 = np.asarray(X_train_distance1)
        X_train_distance2 = np.asarray(X_train_distance2)
        X_train_skeleton = np.asarray(X_train_skeleton)
        y_train = np.asarray(y_train)

        X_test_text = np.asarray(X_test_text)
        X_test_inner_tokens = np.asarray(X_train_inner_tokens)
        X_test_surrounding_tokens = np.asarray(X_train_surrounding_tokens)
        X_test_e1 = np.asarray(X_train_e1)
        X_test_e2 = np.asarray(X_train_e2)
        X_test_distance1 = np.asarray(X_test_distance1)
        X_test_distance2 = np.asarray(X_test_distance2)
        X_test_skeleton = np.asarray(X_test_skeleton)
        y_test = np.asarray(y_test)

        return [X_train_text, X_train_distance1, X_train_distance2, X_train_skeleton, X_train_inner_tokens,
                X_train_surrounding_tokens,
                X_train_e1, X_train_e2], y_train, [X_test_text, X_test_distance1,
                                                   X_test_distance2, X_test_skeleton, X_test_inner_tokens,
                                                   X_test_surrounding_tokens,
                                                   X_test_e1, X_test_e2], y_test

    return get_input_format(train_set2011, test_set2011, target_indices2011), get_input_format(train_set2013, test_set2013, target_indices2013), len(
        word_indices), len(dis_indices)


# 13的offset是卡到最后一个字符所在的位置，而11是卡到最后一个字符的下一个位置
def decodeOffset(offset, version='13'):
    if ';' in offset:
        return tuple([decodeOffset(x) for x in offset.split(';')])
    elif ',' in offset:
        return tuple([decodeOffset(x) for x in offset.split(',')])
    rawOffsets = offset.split('-')
    return (int(rawOffsets[0]), int(rawOffsets[1]) if version == '13' else int(rawOffsets[1]) - 1)


def getIndex(sentence, offset):
    if type(offset[0]) == type(offset[1]) == tuple:
        r = []
        for tup in offset:
            r += getIndex(sentence, tup)
        return r
    start = sentence[:offset[0]].count(' ')
    end = sentence[offset[0]:offset[1]].count(' ') + start
    return list(range(start, end + 1))


# 单纯的把xml的内容解析成实例，不做任何预处理
def get_instances_from_xml_file(xml_file, with_pairid=False, version='13'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    instances = []

    for child in root:
        # print child.tag,
        sentence = child.attrib['text']
        entities = dict()
        pairs = []
        sub_instances = []
        for grandson in child:
            if grandson.tag == 'entity':
                entities[grandson.attrib['id']] = (grandson.attrib['text'], grandson.attrib['charOffset'])
            elif grandson.tag == 'pair':
                pairid = grandson.attrib['id']
                tag = ''
                if version == '13':
                    tag = grandson.attrib['type'] if grandson.attrib['ddi'] == 'true' else '-1'
                elif version == '11':
                    tag = grandson.attrib['interaction']
                    tag = '1' if tag == 'true' else '0'
                    if (grandson.attrib['e1'], grandson.attrib['e2']) in DDIE_2011_gold_standard:
                        tag = DDIE_2011_gold_standard[(grandson.attrib['e1'], grandson.attrib['e2'])]
                pairs.append((grandson.attrib['e1'], grandson.attrib['e2'], tag, pairid))

        sentence = sentence.replace('\n', ' ')
        for e1id, e2id, target, pairid in pairs:
            e1offset = decodeOffset(entities[e1id][1], version=version)
            e2offset = decodeOffset(entities[e2id][1], version=version)

            if with_pairid:
                sub_instances.append((target, sentence, e1offset, e2offset, pairid))
            else:
                sub_instances.append((target, sentence, e1offset, e2offset))
        instances.extend(sub_instances)
    return instances


def extract_sentence(corpus_files, sentence_file):
    sentences = set()
    for corpus in corpus_files:
        for _, sentence, _, _ in read_corpus_new(corpus):
            sentences.add(sentence)

    writer = open(sentence_file, 'w')
    for sentence in sentences:
        writer.write(sentence + '\n')
    writer.close()


def getNewOffset(origin_doc, new_doc, origin_offset, minus1=False):
    while origin_doc[-1] == ' ':
        origin_doc = origin_doc[:-1]
    while new_doc[-1] == ' ':
        new_doc = new_doc[:-1]

    if type(origin_offset[0]) == type(origin_offset[1]) == tuple:
        return tuple([getNewOffset(origin_doc, new_doc, offset) for offset in origin_offset])
    # 游标i卡在origin上，游标j卡在new上
    i = 0
    j = 0
    if minus1:
        origin_offset = origin_offset[0], origin_offset[1] - 1
    new_offset_start, new_offset_end = origin_offset[0], -1
    # i和j的字符必须相同 此时i++ j++  检查i如果是start/end offset 记录j为new start/end offset
    a = origin_doc.lower()
    b = new_doc.lower()
    lena = len(a)
    lenb = len(b)
    while i < lena and j < lenb:

        # print a[i], b[j], '\t', i, j
        if a[i] == b[j]:
            if i == origin_offset[0]:
                new_offset_start = j
            if i == origin_offset[1]:
                new_offset_end = j

            i += 1
            j += 1
            continue
        elif a[i] == ' ':
            i += 1
        elif b[j] == ' ':
            j += 1
        else:
            print(origin_doc)
            print(new_doc)
            print(u'输入的两个句子不是经过extreme tokenization算法得到的，退出')
            return
    if new_offset_end == -1:
        new_offset_end = len(b) - 1
    return new_offset_start, ((new_offset_end + 1) if minus1 else new_offset_end)


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# 把没用的药物实体替换成字符串'drug'
def replace_irrelavent_drugs(instances):
    new_instances = []
    sentence_dic = {}
    for t, sentence, e1, e2 in instances:
        if sentence not in sentence_dic:
            sentence_dic[sentence] = [(e1, e2, t)]
        else:
            sentence_dic[sentence].append((e1, e2, t))
    for sentence in sentence_dic:
        entity_list = []
        for e1, e2, _ in sentence_dic[sentence]:
            if e1 not in entity_list:
                entity_list.append(e1)
            if e2 not in entity_list:
                entity_list.append(e2)
        for e1, e2, t in sentence_dic[sentence]:
            origin_e1 = get_entity_by_range(sentence, e1)
            origin_e2 = get_entity_by_range(sentence, e2)
            token_list = sentence.split(' ')
            for e in entity_list:
                # 直接替换为@
                for offset in e:
                    if offset not in e1 and offset not in e2:
                        token_list[offset] = '@' * len(token_list[offset])
            new_sent = ' '.join(token_list)
            matches = list(re.finditer(r"@+[ @+]+@", new_sent))
            matches.reverse()
            for match in matches:
                match_span = match.span()
                match_range = getIndex(sentence, match_span)
                merges_no = new_sent[match_span[0]: match_span[1]].count(' ')

                e1 = [x - merges_no if x > match_range[-1] else x for x in e1]
                e2 = [x - merges_no if x > match_range[-1] else x for x in e2]
                new_sent = new_sent[:match_span[0]] + 'drug' + new_sent[match_span[1]:]
            new_instances.append((t, new_sent, e1, e2))
            new_e1 = get_entity_by_range(new_sent, e1)
            new_e2 = get_entity_by_range(new_sent, e2)
            assert (origin_e1 == new_e1)
            assert (origin_e2 == new_e2)

    return new_instances


def extreme_fast_tokenize_sentence(sentence, special_chars):
    newSent = []
    for token in sentence.strip().split(' '):
        newToken = ''
        for i in range(len(token)):
            cc = token[i]
            lc = ''
            if len(newToken) != 0:
                lc = newToken[-1]
            if cc in special_chars:
                if i > 0 and lc != ' ':
                    newToken += ' '
                newToken += token[i]
                continue
            if lc in special_chars and cc != ' ':
                newToken += ' '
            newToken += token[i]

        def rm_bracket(newToken):
            newToken = newToken.strip()
            if len(newToken) > 0:
                if newToken[0] == '(' and ')' not in newToken:
                    newToken = newToken[0] + ' ' + rm_bracket(newToken[1:])
                elif newToken[-1] == ')' and '(' not in newToken:
                    newToken = rm_bracket(newToken[:-1]) + ' ' + newToken[-1]
                elif newToken[0] == '(' and newToken[-1] == ')':
                    newToken = newToken[0] + ' ' + rm_bracket(newToken[1:-1]) + ' ' + newToken[-1]
                elif newToken[0] == '\'' and newToken[-1] == '\'':
                    newToken = newToken[0] + ' ' + rm_bracket(newToken[1:-1]) + ' ' + newToken[-1]
            return newToken

        if newToken.count('.') == 1 or len(newToken) > 9:
            newnewToken = ''
            for i in range(len(newToken)):
                cc = newToken[i]
                lc = ''
                if len(newnewToken) != 0:
                    lc = newnewToken[-1]
                if cc == '.':
                    if i > 0 and lc != ' ':
                        newnewToken += ' '
                    newnewToken += newToken[i]
                    continue
                if lc == '.' and cc != ' ':
                    newnewToken += ' '
                newnewToken += newToken[i]
            newToken = newnewToken

        if newToken.count('\'') == 1:
            dotidx = newToken.index('\'')
            newToken = rm_bracket(newToken[:dotidx]) + ' \' ' + rm_bracket(newToken[dotidx + 1:])
        if newToken.count(':') == 1:
            dotidx = newToken.index(':')
            newToken = rm_bracket(newToken[:dotidx]) + ' : ' + rm_bracket(newToken[dotidx + 1:])
        for special_entity in ['TCA', 'Cys']:
            if special_entity in newToken:  # 特殊对待
                idx = newToken.find(special_entity)
                if idx + 3 < len(newToken) and newToken[idx + len(special_entity)] != ' ':
                    newToken = newToken[:idx] + special_entity + ' ' + newToken[idx + 3:]
        newToken = rm_bracket(newToken)
        newSent.append(newToken.strip())
    newSent = ' '.join(newSent)
    while '  ' in newSent:
        newSent = newSent.replace('  ', ' ')
    return newSent


# 11里需要把/ (  )都切开，而13不用
def tokenizeCorpus(instances, lower=False, version='13'):
    new_instances = []
    for target, sentence, d1offset, d2offset in instances:
        if type(d1offset[0]) == tuple:
            origin_entity1 = sentence[d1offset[0][0]:d1offset[0][1] + 1] + ' ' + sentence[
                                                                                 d1offset[1][0]:d1offset[1][1] + 1]
        else:
            origin_entity1 = sentence[d1offset[0]:d1offset[1] + 1]
        if type(d2offset[0]) == tuple:
            origin_entity2 = sentence[d2offset[0][0]:d2offset[0][1] + 1] + ' ' + sentence[
                                                                                 d2offset[1][0]:d2offset[1][1] + 1]
        else:
            origin_entity2 = sentence[d2offset[0]:d2offset[1] + 1]

        tokened_sentence = extreme_fast_tokenize_sentence(sentence,
                                                          special_chars=special_chars_13 if version == '13' else special_chars_11)
        new_d1offset = getNewOffset(sentence, tokened_sentence, d1offset)
        new_d2offset = getNewOffset(sentence, tokened_sentence, d2offset)

        r1 = getIndex(tokened_sentence, new_d1offset)
        r2 = getIndex(tokened_sentence, new_d2offset)
        new_entity1 = get_entity_by_range(tokened_sentence, r1)
        new_entity2 = get_entity_by_range(tokened_sentence, r2)

        origin_entity1 = origin_entity1.lower()
        origin_entity2 = origin_entity2.lower()
        new_entity1 = new_entity1.lower()
        new_entity2 = new_entity2.lower()

        if origin_entity1 != new_entity1 and new_entity1.replace(' ', '') != origin_entity1.replace(' ', ''):
            print('真正实体是：', origin_entity1, d1offset)
            print('改变后却是：', new_entity1)
            print('出现在', sentence)
            print('经过token之后是', tokened_sentence)
        if origin_entity2 != new_entity2 and new_entity2.replace(' ', '') != origin_entity2.replace(' ', ''):
            print('真正实体是：', origin_entity2, d2offset)
            print('改变后却是：', new_entity2)
            print('出现在', sentence)
            print('经过token之后是', tokened_sentence)

        if lower:
            tokened_sentence = tokened_sentence.lower()
        new_instances.append((target, tokened_sentence, r1, r2))
    return new_instances


def charset(instances):
    char_set = set()
    for _, sentence, _, _ in instances:
        for c in sentence:
            char_set.add(c)
    # print(char_set)
    return char_set


# 如果一个token是由单个特殊符号组成，去掉它
def remove_irrelavent_chars(instances):
    new_instances = []
    special_chars = special_chars_11 + ['.']
    for t, sentence, e1r, e2r in instances:
        origin_e1 = get_entity_by_range(sentence, e1r)
        origin_e2 = get_entity_by_range(sentence, e2r)
        token_list = sentence.split(' ')
        for i in range(len(token_list) - 1, -1, -1):
            if len(token_list[i]) == 1 and token_list[i] in special_chars and i not in e1r and i not in e2r:
                e1r = [j - 1 if i < j else j for j in e1r]
                e2r = [j - 1 if i < j else j for j in e2r]
                token_list.pop(i)
        new_sent = ' '.join(token_list)
        new_e1 = get_entity_by_range(new_sent, e1r)
        new_e2 = get_entity_by_range(new_sent, e2r)
        if origin_e1 != new_e1:
            print(origin_e1, new_e1, sentence)
        if origin_e2 != new_e2:
            print(origin_e2, new_e2, sentence)
        new_instances.append((t, new_sent, e1r, e2r))
    return new_instances


def load_gold_11():
    for line in open(ddi_2011_gold):
        e1, e2, label = line[:-1].split()
        DDIE_2011_gold_standard[(e1, e2)] = label


def xml2plain(xmlfolders, plain_file, replace_drug=True, remove_special_chars=False, version='13'):
    if version == '11':
        load_gold_11()
    if isinstance(xmlfolders, str):
        xmlfolders = [xmlfolders]
    instances = []
    bw = open(plain_file, 'w')
    for folder in xmlfolders:
        if folder.endswith('/'):
            folder = folder[:-1]
        for filename in os.listdir(folder):
            instances.extend(get_instances_from_xml_file(folder + '/' + filename, version=version))
    print(u"在tokenization之前，instances的数目是：", len(instances))

    instances = tokenizeCorpus(instances, version=version)
    if replace_drug:
        instances = replace_irrelavent_drugs(instances)
    if remove_special_chars:
        instances = remove_irrelavent_chars(instances)

    print(u"在tokenization之后，instances的数目是：", len(instances))
    for instance in instances:
        bw.write('\t'.join([str(ele) for ele in instance]) + '\n')
    bw.close()
    print(u"写入文件之后，instances的数目是：", file_len(plain_file))


# xml2plain([ddi_db_2013_train_xml_folder], ddi_2013_DB_train, version='13')
# xml2plain([ddi_ml_2013_train_xml_folder], ddi_2013_ML_train, version='13')
# xml2plain([ddi_db_2013_test_xml_folder], ddi_2013_DB_test, version='13')
# xml2plain([ddi_ml_2013_test_xml_folder], ddi_2013_ML_test, version='13')
#
# xml2plain([ddi_2011_train_xml_folder], ddi_2011_train, version='11')
# xml2plain([ddi_2011_test_xml_folder], ddi_2011_test, version='11')

# extract_sentence([ddi_2011_train, ddi_2011_test], ddi_2011_sentences)
# extract_sentence([ddi_2013_DB_train, ddi_2013_ML_train, ddi_2013_DB_test, ddi_2013_ML_test], ddi_2013_sentences)

all_data = load_data()
# summarize_ddi_instance()


In [27]:
#callback部分

import keras
import time
from sklearn.metrics.classification import  confusion_matrix, precision_recall_fscore_support, accuracy_score


class evaluator_2013(keras.callbacks.Callback):
    def __init__(self, model, X_test, X_test_distance1, X_test_distance2, X_test_skeleton, y_test, dim):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.X_test_distance1 = X_test_distance1
        self.X_test_distance2 = X_test_distance2
        self.X_test_skeleton = X_test_skeleton
        self.dim = dim


    def show_DDIE_result(self, y_true, y_pred):
        def precision(TP, FP):
            if TP == FP == 0:
                return 0
            else:
                return TP / (TP + FP)

        def recall(TP, FN):
            if TP == FN == 0:
                return 0
            else:
                return TP / (TP + FN)

        def f_score(p, r):
            if p == r == 0.0:
                return 0.0
            else:
                return 2 * p * r / (p + r)

        cm = confusion_matrix(y_true, y_pred)
        cm_tp_fp_fn = []
        result = [[], [], []]
        for i in range(1, len(cm)):
            tp = cm[i][i]
            fp = sum(cm[:, i]) - tp
            fn = sum(cm[i, :]) - tp
            cm_tp_fp_fn.append((tp, fp, fn))
        micro_p = precision(sum([x[0] for x in cm_tp_fp_fn]), sum([x[1] for x in cm_tp_fp_fn]))
        micro_r = recall(sum([x[0] for x in cm_tp_fp_fn]), sum([x[2] for x in cm_tp_fp_fn]))
        micro_f = f_score(micro_p, micro_r)
        result[0].append(micro_p)
        result[1].append(micro_r)
        result[2].append(micro_f)
        binary_p, binary_r, binary_f, _ = precision_recall_fscore_support(y_true != 0, y_pred != 0, )
        result[0].append(binary_p[1])
        result[1].append(binary_r[1])
        result[2].append(binary_f[1])
        ps = [precision(x[0], x[1]) for x in cm_tp_fp_fn]
        rs = [recall(x[0], x[2]) for x in cm_tp_fp_fn]
        fs = [f_score(ps[i], rs[i]) for i in range(len(ps))]
        for i in range(len(ps)):
            result[0].append(ps[i])
            result[1].append(rs[i])
            result[2].append(fs[i])
        avg_p = sum(ps) / len(ps)
        avg_r = sum(rs) / len(rs)
        result[0].append(avg_p)
        result[1].append(avg_r)
        mavg_f = f_score(avg_p, avg_r)
        result[2].append(mavg_f)
        print("\t\t".join(['CLA', 'DEC', 'MEC', 'EFF', 'ADV', 'INT', 'MAVG']))
        for r in result:
            print('\t'.join(["%.3f" % v for v in r]))
        return mavg_f


    def on_train_begin(self, logs={}):
        self.losses = []
        self.times = []
        self.mafscores = []
        self.dbfscores = []
        self.mlfscores = []
        self.epoch_time_start = time.time()



    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

        # Once the training has ended, display the animation
        y_pred = self.model.predict(
            [self.X_test, self.X_test_distance1, self.X_test_distance2, self.X_test_skeleton]).argmax(axis=1)

        #
        y_true = self.y_test.argmax(axis=1)

        assert (len(y_pred) == 5265 + 451)
        y_pred_DB = y_pred[:5265]
        y_pred_ML = y_pred[5265:]
        y_true_DB = y_true[:5265]
        y_true_ML = y_true[5265:]
        print()
        print("The {}th epoch".format(epoch))
        print("------------------Drug Bank Result------------------")
        DBf = self.show_DDIE_result(y_true_DB, y_pred_DB)
        print("-------------------MEDLINE Result-------------------")
        MLf = self.show_DDIE_result(y_true_ML, y_pred_ML)
        print("-------------------All Set Result-------------------")
        MAf = self.show_DDIE_result(y_true, y_pred)
        # self.model.save_weights('13_weights_dim_' + str(self.dim) + '_epoch_' + str(epoch))
        self.epoch_time_start = time.time()
        self.losses.append(logs.get('loss'))
        self.mafscores.append(MAf)
        self.dbfscores.append(MLf)
        self.mlfscores.append(DBf)


class evaluator_2011(keras.callbacks.Callback):
    def __init__(self, model, X_test, X_test_distance1, X_test_distance2, X_test_skeleton, y_test, dim):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.X_test_distance1 = X_test_distance1
        self.X_test_distance2 = X_test_distance2
        self.X_test_skeleton = X_test_skeleton
        self.dim = dim
        self.iter = iter

    def on_train_begin(self, logs={}):
        self.losses = []
        self.times = []
        self.fscores = []
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        # Once the training has ended, display the animation
        y_pred = self.model.predict(
            [self.X_test, self.X_test_distance1, self.X_test_distance2, self.X_test_skeleton]).argmax(axis=1)
        #
        y_true = self.y_test.argmax(axis=1)
        print()
        print("The {}th epoch".format(epoch))
        print("------------------DDIE2011 Result------------------")
        # y_true = y_true != 0
        # y_pred = y_pred != 0
        print(y_true, sum(y_true))
        print(y_pred, sum(y_pred))
        cm = confusion_matrix(y_true, y_pred)
        TP = cm[1][1]
        FP = cm[0][1]
        TN = cm[0][0]
        FN = cm[1][0]
        binary_p, binary_r, binary_f, _ = precision_recall_fscore_support(y_true, y_pred)
        P = binary_p[1]
        R = binary_r[1]
        F = binary_f[1]
        Acc = accuracy_score(y_true, y_pred)
        print("\t\t".join(['TP', 'FP', 'FN', 'TN', 'P', 'R', 'F', 'Acc']))
        print("\t".join(
            [str(x) + ('\t' if x < 1000 else "") for x in [TP, FP, FN, TN]] + ["%.4f" % v for v in [P, R, F, Acc]]))
        # self.model.save_weights('11_weights_dim_' + str(self.dim) + '_epoch_' + str(epoch))


        self.losses.append(logs.get('loss'))
        self.fscores.append(F)
        self.epoch_time_start = time.time()





In [32]:
# 预训练部分
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM
from keras.engine.topology import Merge
np.random.seed(2)
maxlen = None
dim = 400
batch_size = 32
dropout_ratio = 0.2
start_seed = 0
flag = False
def pretrain(X_train, Y_train, X_test, Y_test, word_embedding_size, dis_embedding_size, version='13'):
    X_train_text, X_train_distance1, X_train_distance2, X_train_skeleton, X_train_inner_tokens, X_train_surrounding_tokens, X_train_e1, X_train_e2 = X_train
    X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, X_test_inner_tokens, X_test_surrounding_tokens, X_test_e1, X_test_e2 = X_test

    print('Pad sequences (samples x time)')
    X_train_text = sequence.pad_sequences(X_train_text, maxlen=maxlen)
    X_train_inner_tokens = sequence.pad_sequences(X_train_inner_tokens, maxlen=maxlen)
    X_test_text = sequence.pad_sequences(X_test_text, maxlen=maxlen)
    X_test_inner_tokens = sequence.pad_sequences(X_test_inner_tokens, maxlen=maxlen)
    X_train_distance1 = sequence.pad_sequences(X_train_distance1, maxlen=maxlen)
    X_train_distance2 = sequence.pad_sequences(X_train_distance2, maxlen=maxlen)
    X_train_skeleton = sequence.pad_sequences(X_train_skeleton, maxlen=maxlen)
    X_test_distance1 = sequence.pad_sequences(X_test_distance1, maxlen=maxlen)
    X_test_distance2 = sequence.pad_sequences(X_test_distance2, maxlen=maxlen)
    X_test_skeleton = sequence.pad_sequences(X_test_skeleton, maxlen=maxlen)
    print('X_train shape:', X_train_text.shape)
    print('X_test shape:', X_test_text.shape)
    # for dim in [300, 400, 500]:
    #     for iter in range(2):
    print('Build model...')
    

    word_input = Sequential()
    word_input.add(
        Embedding(word_embedding_size, EMBEDDING_DIM, dropout=dropout_ratio, trainable=True))

    dis1_input = Sequential()
    dis1_input.add(Embedding(dis_embedding_size, 50, dropout=dropout_ratio))

    dis2_input = Sequential()
    dis2_input.add(Embedding(dis_embedding_size, 50, dropout=dropout_ratio))

    skeleton = Sequential()
    skeleton.add(Embedding(6, 50, dropout=dropout_ratio))
    model = Sequential()
    # model.add(word_input)
    # model.add(merge([word_input, dis1_input, dis2_input, skeleton], mode='concat'))

    model.add(Merge([word_input, dis1_input, dis2_input, skeleton], mode='concat'))
    model.add(LSTM(dim, dropout_W=dropout_ratio, dropout_U=dropout_ratio))

    if version == '13':
        model.add(Dense(5))

    elif version == '11':
        model.add(Dense(2))
    model.add(Activation('softmax'))

    # try using different optimizers and different optimizer configs
    model.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop') #sgd
    model.summary()
    print('Train...')
    
    if version == '13':
        evlt = evaluator_2013(model, X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, Y_test, dim)
    elif version == '11':
        evlt = evaluator_2011(model, X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, Y_test, dim)
        
    model.fit([X_train_text, X_train_distance1, X_train_distance2, X_train_skeleton], Y_train, batch_size=batch_size,
              nb_epoch=20,
              callbacks=[evlt], verbose=0)
    return model

data2011, data2013, word_embedding_size, dis_embedding_size = all_data
X_train2011, Y_train2011, X_test2011, Y_test2011 = data2011
X_train2013, Y_train2013, X_test2013, Y_test2013 = data2013
pretrained_model_11 = pretrain(X_train2011, Y_train2011, X_test2011, Y_test2011,  word_embedding_size, dis_embedding_size, version='11')
pretrained_model_13 = pretrain(X_train2013, Y_train2013, X_test2013, Y_test2013,  word_embedding_size, dis_embedding_size, version='13')

Pad sequences (samples x time)
X_train shape: (23827, 131)
X_test shape: (7026, 110)
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_37 (Embedding)         (None, None, 400)     3624400                                      
____________________________________________________________________________________________________
embedding_38 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_39 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_40 (Embedding)         (None, None, 50)      300                                  

C:\workbench\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Pad sequences (samples x time)
X_train shape: (1787, 94)
X_test shape: (5716, 109)
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_41 (Embedding)         (None, None, 400)     3624400                                      
____________________________________________________________________________________________________
embedding_42 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_43 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_44 (Embedding)         (None, None, 50)      300                                    

In [ ]:
def train(X_train, Y_train, X_test, Y_test, word_embedding_size, dis_embedding_size,  pretrained_model, version='13'):
    X_train_text, X_train_distance1, X_train_distance2, X_train_skeleton, X_train_inner_tokens, X_train_surrounding_tokens, X_train_e1, X_train_e2 = X_train
    X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, X_test_inner_tokens, X_test_surrounding_tokens, X_test_e1, X_test_e2 = X_test

    print('Pad sequences (samples x time)')
    X_train_text = sequence.pad_sequences(X_train_text, maxlen=maxlen)
    X_train_inner_tokens = sequence.pad_sequences(X_train_inner_tokens, maxlen=maxlen)
    X_test_text = sequence.pad_sequences(X_test_text, maxlen=maxlen)
    X_test_inner_tokens = sequence.pad_sequences(X_test_inner_tokens, maxlen=maxlen)
    X_train_distance1 = sequence.pad_sequences(X_train_distance1, maxlen=maxlen)
    X_train_distance2 = sequence.pad_sequences(X_train_distance2, maxlen=maxlen)
    X_train_skeleton = sequence.pad_sequences(X_train_skeleton, maxlen=maxlen)
    X_test_distance1 = sequence.pad_sequences(X_test_distance1, maxlen=maxlen)
    X_test_distance2 = sequence.pad_sequences(X_test_distance2, maxlen=maxlen)
    X_test_skeleton = sequence.pad_sequences(X_test_skeleton, maxlen=maxlen)
    print('X_train shape:', X_train_text.shape)
    print('X_test shape:', X_test_text.shape)
    # for dim in [300, 400, 500]:
    #     for iter in range(2):
    print('Build model...')
    

    word_input = Sequential()
    word_input.add(
        Embedding(word_embedding_size, EMBEDDING_DIM, dropout=dropout_ratio, trainable=True))

    dis1_input = Sequential()
    dis1_input.add(Embedding(dis_embedding_size, 50, dropout=dropout_ratio))

    dis2_input = Sequential()
    dis2_input.add(Embedding(dis_embedding_size, 50, dropout=dropout_ratio))

    skeleton = Sequential()
    skeleton.add(Embedding(6, 50, dropout=dropout_ratio))
    model = Sequential()
    # model.add(word_input)
    # model.add(merge([word_input, dis1_input, dis2_input, skeleton], mode='concat'))

    model.add(Merge([word_input, dis1_input, dis2_input, skeleton], mode='concat'))
    model.add(LSTM(dim, dropout_W=dropout_ratio, dropout_U=dropout_ratio))
    model.layers[0].set_weights(pretrained_model.layers[0].get_weights())
    model.layers[1].set_weights(pretrained_model.layers[1].get_weights())
    if version == '13':
        model.add(Dense(5))

    elif version == '11':
        model.add(Dense(2))
    model.add(Activation('softmax'))

    # try using different optimizers and different optimizer configs
    model.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop') #sgd
    model.summary()
    print('Train...')
    
    if version == '13':
        evlt = evaluator_2013(model, X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, Y_test, dim)
    elif version == '11':
        evlt = evaluator_2011(model, X_test_text, X_test_distance1, X_test_distance2, X_test_skeleton, Y_test, dim)
        
    model.fit([X_train_text, X_train_distance1, X_train_distance2, X_train_skeleton], Y_train, batch_size=batch_size,
              nb_epoch=20,
              callbacks=[evlt], verbose=0)
    return model

train(X_train2011, Y_train2011, X_test2011, Y_test2011,  word_embedding_size, dis_embedding_size,pretrained_model_13,  version='11')


Pad sequences (samples x time)
X_train shape: (23827, 131)
X_test shape: (7026, 110)
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_49 (Embedding)         (None, None, 400)     3624400                                      
____________________________________________________________________________________________________
embedding_50 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_51 (Embedding)         (None, None, 50)      12800                                        
____________________________________________________________________________________________________
embedding_52 (Embedding)         (None, None, 50)      300                                  

C:\workbench\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



The 0th epoch
------------------DDIE2011 Result------------------
[1 0 1 ... 0 0 0] 755
[0 0 0 ... 0 0 0] 10
TP		FP		FN		TN		P		R		F		Acc
7		3		748		6268	0.7000	0.0093	0.0183	0.8931

The 1th epoch
------------------DDIE2011 Result------------------
[1 0 1 ... 0 0 0] 755
[0 0 0 ... 0 0 0] 355
TP		FP		FN		TN		P		R		F		Acc
236		119		519		6152	0.6648	0.3126	0.4252	0.9092

The 2th epoch
------------------DDIE2011 Result------------------
[1 0 1 ... 0 0 0] 755
[0 0 0 ... 0 0 0] 633
TP		FP		FN		TN		P		R		F		Acc
366		267		389		6004	0.5782	0.4848	0.5274	0.9066

The 3th epoch
------------------DDIE2011 Result------------------
[1 0 1 ... 0 0 0] 755
[0 0 0 ... 0 0 0] 367
TP		FP		FN		TN		P		R		F		Acc
261		106		494		6165	0.7112	0.3457	0.4652	0.9146

The 4th epoch
------------------DDIE2011 Result------------------
[1 0 1 ... 0 0 0] 755
[0 0 0 ... 0 0 0] 977
TP		FP		FN		TN		P		R		F		Acc
504		473		251		5798	0.5159	0.6675	0.5820	0.8970

The 5th epoch
------------------DDIE2011 Result-----------------